In [4]:
!pip install selenium

distributed 1.21.8 requires msgpack, which is not installed.
You are using pip version 10.0.1, however version 19.1.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [5]:
from bs4 import BeautifulSoup 
from selenium import webdriver 
from selenium.webdriver.common.keys import Keys 
from datetime import datetime 
import pandas as pd 
import time 
import re
from selenium.common.exceptions import NoSuchElementException

In [1]:
#파일로 저장한 동영상 링크를 불러오기
file1 = open("url2 (1).txt","r")
url=[]
while True:
    l=file1.readline()
    if l == "":
        break
    url.append(l)
file1.close()

In [6]:
n_url=[]
for i in url:
    i= re.sub("\n","",i)
    n_url.append(i)

유튜브 환경 특성상 bs4 혹은 API를 사용하기 어려워 selenium을 활용

In [179]:
#최초 저장시 헤더를 넣기 위한 체크
data_dic={
    'link':[],
    'title':[],
    'view':[],
    'good':[],
    'bad':[],
    'description':[],
    'date':[],
    'number of comment':[]
}

comment={
    'link':[],
    'comment':[],
    'date':[],
    'comment date':[]
}
driver = webdriver.Chrome('chromedriver')
for lnk in n_url:
    driver.get(lnk)
    num_page = 50
    while num_page:
        body=driver.find_element_by_css_selector("body")
        body.send_keys(Keys.PAGE_DOWN)
        time.sleep(1)
        num_page -= 1
    time.sleep(1)
    data_dic['link'].append(lnk)
    data_dic['title'].append(driver.find_element_by_css_selector('#container > h1 > yt-formatted-string').text)
    data_dic['view'].append(driver.find_element_by_css_selector('#count > yt-view-count-renderer > span.view-count.style-scope.yt-view-count-renderer').text)
    data_dic['good'].append(driver.find_element_by_css_selector('#top-level-buttons > ytd-toggle-button-renderer:nth-child(1)').text)
    data_dic['bad'].append(driver.find_element_by_css_selector('#top-level-buttons > ytd-toggle-button-renderer:nth-child(2)').text)
    data_dic['description'].append(driver.find_element_by_css_selector('#description > yt-formatted-string').text)
    data_dic['date'].append(driver.find_element_by_css_selector('#upload-info > span').text)
    try:
        numco=driver.find_element_by_css_selector('#count > yt-formatted-string').text
        data_dic['number of comment'].append(numco)
    except NoSuchElementException:
        data_dic['number of comment'].append('0')
    for i in driver.find_elements_by_css_selector('#content-text') :
        comment['link'].append(lnk)
        try:
            comment['comment'].append(i.text)
        except NoSuchElementException:
            comment['comment'].append("")
    for j in driver.find_elements_by_css_selector('#header-author > yt-formatted-string > a'):
        comment['date'].append(driver.find_element_by_css_selector('#upload-info > span').text)
        try:
            comment['comment date'].append(j.text)
        except NoSuchElementException:
            comment['comment date'].append('NA')
#딕셔너리에 넣은 정보를 데이터 프레임으로 변환
df = pd.DataFrame.from_dict(data_dic)
df2= pd.DataFrame.from_dict(comment)

df.columns = ['link','title','view','good','bad','description','date','number of comment']
df2.columns = ['link', 'comment','comment date','date']
df.to_csv('190513_data.csv', index=False, encoding='utf-8-sig')
df2.to_csv('190513_comment.csv', index=False, encoding='utf-8-sig')
print("수집 완료~!!!")

,link,title,view,good,bad,description,date,number of comment
0,https://www.youtube.com/watch?v=A-Qa8Uyg4XY,One Brand Makeup Tutorial INNISFREE (Eng& Baha...,"조회수 269,429회",5.3천,56,"Hi guys, I'm back with another tutorial video....",게시일: 2016. 7. 6.,댓글 113개
1,https://www.youtube.com/watch?v=XfQN-VY5vk8,[TESTERKOREA] INNISFREE The Green Tea Seed Oil,"조회수 12,161회",62,10,INNISFREE The Green Tea Seed Oil 30ml\n\nhttp:...,게시일: 2016. 10. 27.,댓글 3개
2,https://www.youtube.com/watch?v=c-55rPMdxRw,Does it work? Innisfree Wine Jelly Softner - H...,"조회수 17,604회",120,67,Subscribe for Videos every week: ▶https://www....,게시일: 2017. 3. 25.,댓글 59개
3,https://www.youtube.com/watch?v=HOuuzVz1T2A,"[이응] 이니스프리 마이 블러셔, 마이 아이섀도우 리뷰","조회수 62,961회",418,65,오늘은 이니스프리 멤버쉽데이에 구매했던 화장품들을 소개해드릴거에요!! 아주아주 퀄리...,게시일: 2017. 5. 28.,댓글 146개
4,https://www.youtube.com/watch?v=YL1-3bvwn64,여자친구(GFriend)'s Sowon and Yerin 이니스프리(Innisfre...,"조회수 1,088회",75,1,▶CLICK HERE FOR MORE KOREAN CFs WITH YOUR FAVO...,게시일: 2019. 4. 27.,댓글 1개
5,https://www.youtube.com/watch?v=l_H6CsOCZ_8,Lần Đầu Tiên Đến Innisfree Hà Nội ♡ Innisfree ...,"조회수 278,069회",8.7천,112,Yeah!!!! Không ai đánh thuế giấc mơ nên chúng ...,게시일: 2018. 7. 4.,"댓글 4,799개"
6,https://www.youtube.com/watch?v=xzSZjzTHusI,INNISFREE MAKEUP REVIEW | Korean One Brand Mak...,"조회수 176,970회",2.5천,58,Another Korean One Brand Tutorial... this week...,게시일: 2016. 5. 4.,댓글 178개
7,https://www.youtube.com/watch?v=JyWtEgMy1Cc,Spring warm-tone makeup with innisfree (With s...,"조회수 878,024회",1.9만,260,* 본 영상은 이니스프리 마이 팔레트와 함께 합니다. *\n\n \n\nPerson...,게시일: 2018. 3. 1.,댓글 544개
8,https://www.youtube.com/watch?v=quY34p7fm18,[EDITOR REVIEW & HOW-TO] Innisfree Jeju Volcan...,"조회수 54,651회",350,20,Full Review: https://kbeautynow.com/2016/06/10...,게시일: 2016. 6. 9.,댓글 23개


간단한 전처리

In [122]:
for i in range(df.shape[0]):
    df['view'][i]=re.sub('조회수', '', df['view'][i])
    df['view'][i]=re.sub('회', '', df['view'][i])
    df['view'][i]=re.sub(',', '', df['view'][i])
df['view']

0       1167633
1         40214
2         58448
3        145379
4         46199
5        171300
6        143075
7        531520
8         20188
9          1090
10      1532243
11       135222
12        20298
13       551846
14       307474
15       210640
16        51737
17       102606
18       143881
19        42900
20         7107
21        40856
22        29642
23        14333
24        15881
25        43762
26        69760
27        13213
28       161809
29       102569
         ...   
224      125593
225      268926
226       11671
227      169950
228       30457
229      304873
230      490831
231       27716
232      162174
233       12117
234      131321
235      876853
236     1067080
237      183365
238       69027
239       32480
240        5165
241      117450
242       94205
243        2830
244       56228
245       57667
246        9888
247       48487
248       12669
249       43884
250      146187
251      178175
252      231789
253       14305
Name: view, Length: 254,

In [123]:
#str 좋아요 확인
print(sum((df['good']=='좋아요')*1))
print(sum((df['good']==0)*1))

3
0


In [124]:
goodnum=[]
for i in range(df.shape[0]):
    if df['good'][i][-1] == '천':
        goodnum.append(int(float(df['good'][i][:-1])*1000))
    elif df['good'][i][-1] == '만':
        goodnum.append(int(float(df['good'][i][:-1])*10000))
    elif df['good'][i] == '좋아요':
        goodnum.append(0)
    else:
        goodnum.append(int(df['good'][i]))
df['good']=goodnum

In [8]:
#키워드 검색으로 나오는 동영상의 링크를 크롤링하여 파일로 저장
file1 = open("url.txt","w")
for line in url:
    file1.writelines(line + '\n')
file1.close()